# DataSet APIの動作確認

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf

In [2]:
sess = tf.InteractiveSession()

## Util function
- 画像リストファイル（CSV形式）から画像データセットを作成するための、補助関数

In [3]:
def read_csv(filename):
    """
    データファイルの定義ファイルを読み込むgeneratorを作る
    """
    with open(filename, 'r') as f:
        for line in f.readlines():
            record = line.rstrip().split(',')
            image_file = record[0]
            label = int(record[1])
            yield image_file, label

def read_image(image_file, label):
    contents = tf.read_file(image_file)
    image = tf.image.decode_image(contents)  # 画像データを[0,1)の範囲に変換
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

In [4]:
file_path = 'image_list_test.csv'
n_batch = 5

## one_shot_iterator
- 入力データを一通り回すための最もシンプルなイテレータ

In [5]:
def get_input_op(f_name):
    """
    データ入力関数
    """
    def generator(): return read_csv(f_name)  # generator

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_types=(tf.string, tf.int64),
        output_shapes=(tf.TensorShape([]), tf.TensorShape([])))
    dataset = dataset.map(read_image)
    dataset = dataset.shuffle(10000)
    dataset = dataset.batch(n_batch)
    iterator = dataset.make_one_shot_iterator()
    images, labels = iterator.get_next()
    return images, labels

In [6]:
test_images, test_labels = get_input_op(file_path)

try:
    while True:
        xs, ys = sess.run([test_images, test_labels])
        print('{}, {}'.format(xs.shape, ys))
except tf.errors.OutOfRangeError:
    pass

(5, 275, 370, 3), [1 0 1 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]


## Initializable Iterator
- データセットの状態を初期化して、一巡した後に明示的に次のepochを指定できる
- iteratorを初期化するときに、一つ以上のplaceholderを使ってデータセットの定義をパラメータ化できる

### Simple Example

In [7]:
range_param = tf.placeholder(tf.int64) # 初期化するパラメータのplaceholderを指定
dataset = tf.data.Dataset.range(range_param)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

sess.run(iterator.initializer, feed_dict={range_param: 3})
try:
    while True:
        print(sess.run(next_element))
except tf.errors.OutOfRangeError:
    pass

print('next dataset')

sess.run(iterator.initializer, feed_dict={range_param: 5})
try:
    while True:
        print(sess.run(next_element))
except tf.errors.OutOfRangeError:
    pass

0
1
2
next dataset
0
1
2
3
4


### Read Image List File
- ToDo
    - file_pathをパラメータとしてiteratorの初期化ができないかしら(tryしたけど失敗した)

In [8]:
class DataSetInitializable:
    def __init__(self, file_path, n_batch):
        self.file_path = file_path
        self.n_batch = n_batch

    def get_input_op(self):
        """
        データ入力関数
        """
        def generator():
            return read_csv(self.file_path)  # generator
        dataset = tf.data.Dataset.from_generator(
            generator,
            output_types=(tf.string, tf.int64),
            output_shapes=(tf.TensorShape([]), tf.TensorShape([])))\
            .map(read_image)
        dataset = dataset.shuffle(10000)
        dataset = dataset.batch(self.n_batch)
        iterator = dataset.make_initializable_iterator()
        images, labels = iterator.get_next()
        self.iterator = iterator
        return images, labels

In [9]:
dataset = DataSetInitializable(file_path=file_path, n_batch=n_batch)
images, labels = dataset.get_input_op()

sess.run(dataset.iterator.initializer) # 次のepochを回す際にもinitializerでiteratorの初期化する
try:
    while True:
        xs, ys = sess.run([images, labels])
        print('{}, {}'.format(xs.shape, ys))
except tf.errors.OutOfRangeError:
    pass

(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 1 0]
(5, 275, 370, 3), [0 0 1 0 0]
(5, 275, 370, 3), [0 0 0 0 0]


## reinitializable Iterator
- 複数のDatasetオブジェクトからIteratorを生成できる
- 訓練データとテストデータを切り替えることができる

### Simple Example

In [10]:
# 二つの異なるdatasetを作る
# *** 訓練データ:0~99の数値にランダムノイズを加えたデータ
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
# *** 評価データ:0~49の数値列
validation_dataset = tf.data.Dataset.range(50)

# Iteratorの定義
# ** データ構造をdatasetが持つプロパティ(output_types, output_shapes)で定義する
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                           training_dataset.output_shapes)
next_element = iterator.get_next()

# iteratorをそれぞれのデータセットで初期化するためのoperationの定義
training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

# 計算グラフの実行
sess.run(training_init_op)
try:
    while True:
        print(sess.run(next_element))
except tf.errors.OutOfRangeError:
    pass

print('next dataset')

sess.run(validation_init_op)
try:
    while True:
        print(sess.run(next_element))
except tf.errors.OutOfRangeError:
    pass


1
6
9
7
0
14
5
-2
-1
12
0
1
14
16
8
23
9
26
27
22
18
19
24
24
19
30
17
17
20
30
23
27
26
38
38
28
43
45
43
40
43
37
36
37
36
40
47
42
49
41
57
56
45
57
47
54
50
56
63
66
69
56
52
61
59
68
63
72
77
60
63
67
73
71
78
82
68
73
76
87
81
75
75
84
89
84
93
91
80
89
84
83
98
97
99
100
103
92
90
97
next dataset
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


### Read Image List File

In [11]:
class DataSetReInitializable:
    def __init__(self, file_path, n_batch):
        self.file_path = file_path
        self.n_batch = n_batch

    def get_input_dataset(self):
        def generator():
            return read_csv(self.file_path)  # generator
        dataset = tf.data.Dataset.from_generator(
            generator,
            output_types=(tf.string, tf.int64),
            output_shapes=(tf.TensorShape([]), tf.TensorShape([])))\
            .map(read_image)
        dataset = dataset.shuffle(10000)
        dataset = dataset.batch(self.n_batch)
        return dataset

In [12]:
# dataset
train_ds = DataSetReInitializable(file_path='image_list_train.csv', n_batch=n_batch)
train_dataset = train_ds.get_input_dataset()
valid_ds = DataSetReInitializable(file_path='image_list_test.csv', n_batch=n_batch)
valid_dataset = valid_ds.get_input_dataset()


In [13]:
# Iteratorの定義
# ** データ構造をdatasetが持つプロパティ(output_types, output_shapes)で定義する
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
next_element = iterator.get_next()

# iteratorをそれぞれのデータセットで初期化するためのoperationの定義
training_init_op = iterator.make_initializer(train_dataset)
validation_init_op = iterator.make_initializer(valid_dataset)

# 計算グラフの実行
sess.run(training_init_op)
try:
    while True:
        images, labels = sess.run(next_element)
        print('{}, {}'.format(images.shape, labels))
except tf.errors.OutOfRangeError:
    pass

print('next dataset')

sess.run(validation_init_op)
try:
    while True:
        images, labels = sess.run(next_element)
        print('{}, {}'.format(images.shape, labels))
except tf.errors.OutOfRangeError:
    pass


(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 1 1 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 1 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 1 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(4, 275, 370, 3), [0 0 0 0]
next dataset
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [0 0 0 0 0]
(5, 275, 370, 3), [1 0 0 0 0]
(5, 275, 370, 3), [0 1 0 0 0]
